# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 21 2023 9:23AM,261560,16,TASA-289786,TASA USA Inc.,Released
1,Apr 21 2023 9:18AM,261559,20,9401529,"Andelyn Biosciences, Inc.",Executing
2,Apr 21 2023 9:16AM,261557,10,PRF-53396-BO,Bio-PRF,Released
3,Apr 21 2023 9:16AM,261557,10,PRF-54100-BO,Bio-PRF,Released
4,Apr 21 2023 9:16AM,261557,10,PRF-54382-BO,Bio-PRF,Released
5,Apr 21 2023 9:16AM,261557,10,PRF-54848-BO,Bio-PRF,Released
6,Apr 21 2023 9:16AM,261557,10,PRF-55504-BO,Bio-PRF,Released
7,Apr 21 2023 9:16AM,261557,10,PRF-55510-BO,Bio-PRF,Released
8,Apr 21 2023 9:16AM,261557,10,PRF-55552-BO,Bio-PRF,Released
9,Apr 21 2023 9:16AM,261557,10,PRF-56200,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,261556,Released,6
17,261557,Released,15
18,261558,Released,1
19,261559,Executing,1
20,261560,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261556,NaN,6.0
261557,NaN,15.0
261558,NaN,1.0
261559,1.0,NaN
261560,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261484,0.0,1.0
261505,1.0,0.0
261514,0.0,2.0
261520,0.0,1.0
261534,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261484,0,1
261505,1,0
261514,0,2
261520,0,1
261534,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261484,0,1
1,261505,1,0
2,261514,0,2
3,261520,0,1
4,261534,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261484,,1
1,261505,1,
2,261514,,2
3,261520,,1
4,261534,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 21 2023 9:23AM,261560,16,TASA USA Inc.
1,Apr 21 2023 9:18AM,261559,20,"Andelyn Biosciences, Inc."
2,Apr 21 2023 9:16AM,261557,10,Bio-PRF
17,Apr 21 2023 9:14AM,261558,10,Bio-PRF
18,Apr 21 2023 9:11AM,261556,10,"Digital Brands, LLC"
24,Apr 21 2023 9:10AM,261555,19,"Emersa Waterbox, LLC"
25,Apr 21 2023 9:08AM,261554,19,"Emersa Waterbox, LLC"
27,Apr 21 2023 9:08AM,261553,10,Hush Hush
50,Apr 21 2023 9:03AM,261551,10,"Citieffe, Inc."
51,Apr 21 2023 9:02AM,261550,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 21 2023 9:23AM,261560,16,TASA USA Inc.,,1
1,Apr 21 2023 9:18AM,261559,20,"Andelyn Biosciences, Inc.",1,
2,Apr 21 2023 9:16AM,261557,10,Bio-PRF,,15
3,Apr 21 2023 9:14AM,261558,10,Bio-PRF,,1
4,Apr 21 2023 9:11AM,261556,10,"Digital Brands, LLC",,6
5,Apr 21 2023 9:10AM,261555,19,"Emersa Waterbox, LLC",,1
6,Apr 21 2023 9:08AM,261554,19,"Emersa Waterbox, LLC",,2
7,Apr 21 2023 9:08AM,261553,10,Hush Hush,,23
8,Apr 21 2023 9:03AM,261551,10,"Citieffe, Inc.",,1
9,Apr 21 2023 9:02AM,261550,10,Emerginnova,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 9:23AM,261560,16,TASA USA Inc.,1,
1,Apr 21 2023 9:18AM,261559,20,"Andelyn Biosciences, Inc.",,1
2,Apr 21 2023 9:16AM,261557,10,Bio-PRF,15,
3,Apr 21 2023 9:14AM,261558,10,Bio-PRF,1,
4,Apr 21 2023 9:11AM,261556,10,"Digital Brands, LLC",6,
5,Apr 21 2023 9:10AM,261555,19,"Emersa Waterbox, LLC",1,
6,Apr 21 2023 9:08AM,261554,19,"Emersa Waterbox, LLC",2,
7,Apr 21 2023 9:08AM,261553,10,Hush Hush,23,
8,Apr 21 2023 9:03AM,261551,10,"Citieffe, Inc.",1,
9,Apr 21 2023 9:02AM,261550,10,Emerginnova,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 9:23AM,261560,16,TASA USA Inc.,1,
1,Apr 21 2023 9:18AM,261559,20,"Andelyn Biosciences, Inc.",,1
2,Apr 21 2023 9:16AM,261557,10,Bio-PRF,15,
3,Apr 21 2023 9:14AM,261558,10,Bio-PRF,1,
4,Apr 21 2023 9:11AM,261556,10,"Digital Brands, LLC",6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 9:23AM,261560,16,TASA USA Inc.,1.0,NaN
1,Apr 21 2023 9:18AM,261559,20,"Andelyn Biosciences, Inc.",NaN,1.0
2,Apr 21 2023 9:16AM,261557,10,Bio-PRF,15.0,NaN
3,Apr 21 2023 9:14AM,261558,10,Bio-PRF,1.0,NaN
4,Apr 21 2023 9:11AM,261556,10,"Digital Brands, LLC",6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 9:23AM,261560,16,TASA USA Inc.,1.0,0.0
1,Apr 21 2023 9:18AM,261559,20,"Andelyn Biosciences, Inc.",0.0,1.0
2,Apr 21 2023 9:16AM,261557,10,Bio-PRF,15.0,0.0
3,Apr 21 2023 9:14AM,261558,10,Bio-PRF,1.0,0.0
4,Apr 21 2023 9:11AM,261556,10,"Digital Brands, LLC",6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2092394,50.0,0.0
15,261514,2.0,0.0
16,784652,3.0,0.0
19,1046127,5.0,0.0
20,523105,3.0,1.0
21,261505,0.0,1.0
22,523092,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2092394,50.0,0.0
1,15,261514,2.0,0.0
2,16,784652,3.0,0.0
3,19,1046127,5.0,0.0
4,20,523105,3.0,1.0
5,21,261505,0.0,1.0
6,22,523092,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,50.0,0.0
1,15,2.0,0.0
2,16,3.0,0.0
3,19,5.0,0.0
4,20,3.0,1.0
5,21,0.0,1.0
6,22,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,50.0
1,15,Released,2.0
2,16,Released,3.0
3,19,Released,5.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Executing,0.0,0.0,0.0,0.0,1.0,1.0,1.0
Released,50.0,2.0,3.0,5.0,3.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Executing,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,Released,50.0,2.0,3.0,5.0,3.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Executing,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,Released,50.0,2.0,3.0,5.0,3.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()